In [1]:
import csv
import pandas as pd
from wikimedia import searchEntity, searchObjWProperty, searchProperty
from dateutil.parser import parse
import operator

In [2]:
limit = 5
threshold = 0

def is_date(string):
    try: 
        parse(string)
        return True
    except ValueError:
        return False
    
def ranking(candidateList, goal, flag, propertyLbl):
    scoreList = [0] * len(candidateList)
    for i in range(len(candidateList)):
#         if(not flag):
#             results = searchObjWProperty(candidateList[i]['id'], 'P31')
#             if(len(results['results']['bindings']) > 0):
#                 temp = False
#                 for elem in results['results']['bindings']:
#                     if(elem['itemLabel']['value'].lower() == propertyLbl.lower()):
#                         temp = True
#                         break
#                 if(not temp):
#                     scoreList[i] = scoreList[i] - 5
        results = searchObjWProperty(candidateList[i]['id'], 'P17')
        if(len(results['results']['bindings']) > 0):
            if(results['results']['bindings'][0]['itemLabel']['value'] == 'Indonesia'):
                scoreList[i] = scoreList[i] + 1
        
        lblDis = lDistance(goal, candidateList[i]['label'].lower())
        aliDis = 10000000
        if('aliases' in candidateList[i]):
            aliDis = lDistance(goal, candidateList[i]['aliases'][0].lower())
        
        scoreList[i] = scoreList[i] - min(lblDis, aliDis)
    
    maxScore = max(scoreList)
    if(maxScore < threshold):
        return ''

    idx = scoreList.index(maxScore)
    return candidateList[idx]['id']

def searchID(flag, cell, rowHead):
    json = searchEntity(cell.lower(), limit)['search']
    id = ''
    if(len(json) > 1):
        id = ranking(json, cell.lower(), flag, rowHead)
    elif(len(json) == 1):
        id = json[0]['id']
    
    if(id == '' and flag):
        id = 'QNew'
    elif(id == ''):
        id = 'QNPNew'
        
    return id

def makeDatatypeMap(header_list, df):
    dtMap = []
    for elem in df.loc[0, header_list]:
        try:
            if(elem.replace('.','').replace(',', '').replace('-', '').replace(' ', '').isdigit()):
                dtMap.append(1)
            elif('jl.' in elem.lower()):
                dtMap.append(1)
            elif(is_date(elem)):
                dtMap.append(1)
            else:
                dtMap.append(0)
        except AttributeError:
            dtMap.append(1)
    return dtMap

def lDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [3]:
searchObjWProperty('Q9842', 'P31')['results']['bindings']

[]

In [6]:
searchID(False, "SLB", "Jenjang Sekolah")

'Q131499'

In [3]:
mapping = {}
mapping['kecamatan'] = 'P131(2)'
mapping['kelurahan'] = 'P131'
mapping['jenjang sekolah'] = 'P31'
mapping['alamat sekolah'] = 'P6375'
mapping['alamat'] = 'P6375'
mapping['jumlah siswa'] = 'P2196'
mapping['telp sekolah'] = 'P1329'
mapping['lokasi geografis'] = 'P625'
mapping['nama sekolah'] = 'P31'


In [4]:
df = pd.read_csv("data/SMKRawanBanjir.csv", encoding='latin-1')
protagonist = 'Nama Sekolah'
df

,No.,Nama Sekolah,Alamat,Kelurahan,Kecamatan,Jumlah Siswa,Jumlah Guru,Telp Sekolah,Kondisi Lingkungan,Lokasi Geografis
0,1,SMK Analis Kesehatan Tunas Harapan,Jl. Bendera Raya - Intisari 3,Kalisari,Pasar Rebo,350,39.0,021-8720108,Banjir,"-6.331491, 106.858711"
1,2,SMK Bhakti,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Wijaya Kusuma,Grogol Petamburan,30,13.0,82114486336,Banjir,"-6.161085, 106.773727"
2,3,SMK Bunga Nusa I,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Cengkareng Barat,Cengkareng,60,10.0,(021)5447107,Banjir,"-6.141474, 106.725876"
3,4,SMK Dewi Sartika,Jl. Kebon Nanas Utara II/23,Cipinang Cempedak,Jatinegara,295,21.0,021-85914453,Banjir,"1.000000, 2.000000"
4,5,SMK Hang Tuah I Jakarta,Jl. Tabah Raya Komp. TNI AL,Kelapa Gading Barat,Kelapa Gading,512,28.0,(021)4535140,Banjir,"-6.158968, 106.885063"
5,6,SMK Ibu Pertiwi 1 Jakarta,Jl. Let. Jend. S. Parman No. 69,Slipi,Palmerah,0,14.0,(021)70624646,Banjir,"-6.193888, 106.797432"
6,7,SMK Islam Fatahillah Jakarta,Jl. Keutamaan No. 89,Krukut,Taman Sari,144,14.0,(021) 6344508,Banjir,"-6.156000, 106.814003"
7,8,SMK Karya Bahari,Jl. Mindi No. 36,Lagoa,K o j a,55,13.0,43909933,Banjir,"-6.113743, 106.916031"
8,9,SMK Kesatuan,Jl. Kesatuan No.1 kp.Klingkit,Rawa Buaya,Cengkareng,306,18.0,(021)58300174,Banjir,"-6.156400, 106.728104"
9,10,SMK Kimia Tunas Harapan,Jl. Bendera Raya / Intisari III No.17,Kalisari,Pasar Rebo,209,22.0,(021)8728766,Banjir,"-6.330335, 106.858643"


In [5]:
header_list = list(df)
header_list.remove('No.')
header_list

['Nama Sekolah',
 'Alamat',
 'Kelurahan',
 'Kecamatan',
 'Jumlah Siswa',
 'Jumlah Guru',
 'Telp Sekolah',
 'Kondisi Lingkungan',
 'Lokasi Geografis']

In [6]:
dtMap = makeDatatypeMap(header_list, df)
dtMap

[0, 1, 0, 0, 1, 1, 1, 0, 1]

In [13]:
convertMap = {}
finalMap2 = {}
print("Start processing Table")
for header in header_list:
    print("Processing {} Column".format(header))
    columnList = []
    for cell in df[header]:
        if(dtMap[header_list.index(header)]):
            id = cell
        else:
            temp = "{}{}".format(header, cell)
            if(temp in convertMap):
                id = convertMap[temp]
            else:
                id = searchID(protagonist == header, cell, header)
                convertMap[temp] = id
        columnList.append(id)
    print("finished processing {} datas".format(len(columnList)))
    finalMap2[header] = columnList

print("All task Finished")

Start processing Table
Processing Nama Sekolah Column
finished processing 25 datas
Processing Alamat Column
finished processing 25 datas
Processing Kelurahan Column
finished processing 25 datas
Processing Kecamatan Column
finished processing 25 datas
Processing Jumlah Siswa Column
finished processing 25 datas
Processing Jumlah Guru Column
finished processing 25 datas
Processing Telp Sekolah Column
finished processing 25 datas
Processing Kondisi Lingkungan Column
finished processing 25 datas
Processing Lokasi Geografis Column
finished processing 25 datas
All task Finished


In [12]:
searchProperty('Q60610068', 'Q6472418')

{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/prop/direct/P131'},
    'itemLabel': {'type': 'literal',
     'value': 'http://www.wikidata.org/prop/direct/P131'}}]}}

In [16]:
new_df2 = pd.DataFrame.from_dict(finalMap2)
new_df2

,Nama Sekolah,Alamat,Kelurahan,Kecamatan,Jumlah Siswa,Jumlah Guru,Telp Sekolah,Kondisi Lingkungan,Lokasi Geografis
0,QNew,Jl. Bendera Raya - Intisari 3,Q12488768,Q193374,350,39.0,021-8720108,Q8068,"-6.331491, 106.858711"
1,QNew,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Q7999779,Q192723,30,13.0,82114486336,Q8068,"-6.161085, 106.773727"
2,QNew,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Q5058847,Q192714,60,10.0,(021)5447107,Q8068,"-6.141474, 106.725876"
3,Q31778996,Jl. Kebon Nanas Utara II/23,Q12479264,Q193323,295,21.0,021-85914453,Q8068,"1.000000, 2.000000"
4,QNew,Jl. Tabah Raya Komp. TNI AL,Q2806467,Q193545,512,28.0,(021)4535140,Q8068,"-6.158968, 106.885063"
5,QNew,Jl. Let. Jend. S. Parman No. 69,Q13097193,Q192745,0,14.0,(021)70624646,Q8068,"-6.193888, 106.797432"
6,QNew,Jl. Keutamaan No. 89,Q12492672,Q192748,144,14.0,(021) 6344508,Q8068,"-6.156000, 106.814003"
7,Q60610068,Jl. Mindi No. 36,Q6472418,QNPNew,55,13.0,43909933,Q8068,"-6.113743, 106.916031"
8,Q20427513,Jl. Kesatuan No.1 kp.Klingkit,Q4201202,Q192714,306,18.0,(021)58300174,Q8068,"-6.156400, 106.728104"
9,Q12510798,Jl. Bendera Raya / Intisari III No.17,Q12488768,Q193374,209,22.0,(021)8728766,Q8068,"-6.330335, 106.858643"


In [18]:
propMap = {}
for header in header_list:
    if(header != protagonist):
        propCount = {}
        for x in range(len(new_df2[header])):
            if(new_df2[header][x] != 'QNPNew' and new_df2[protagonist][x] != 'QNew'):
                print(new_df2[header][x])
                print(new_df2[protagonist][x])
                result = searchProperty(new_df2[protagonist][x], new_df2[header][x])
                if(len(result['results']['bindings']) > 0):
                    for prop in result['results']['bindings']:
                        temp = prop.split('/')
                        if(temp in propCount):
                            propCount[temp] = propCount[temp] + 1
                        else:
                            propCount[temp] = 1
        propMap[header] = max(stats.items(), key=operator.itemgetter(1))[0]

Jl. Kebon Nanas Utara II/23
Q31778996


QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'SPARQL-QUERY: queryStr=\n    SELECT ?item ?itemLabel\n    WHERE\n    {\n        wd:Q31778996 ?item wd:Jl. Kebon Nanas Utara II/23 .\n        SERVICE wikibase:label { bd:serviceParam wikibase:language "id" }\n    }\n    \njava.util.concurrent.ExecutionException: org.openrdf.query.MalformedQueryException: Lexical error at line 5, column 40.  Encountered: " " (32), after : "Kebon"\n\tat java.util.concurrent.FutureTask.report(FutureTask.java:122)\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:206)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:293)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:679)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServlet.doGet(RESTServlet.java:240)\n\tat com.bigdata.rdf.sail.webapp.Mu

In [42]:
convertMap = {}
finalMap = {}
print("Start processing Table")
for header in header_list:
    if(header.lower() in mapping):
        print("Processing {} Column".format(header))
        columnList = []
        for cell in df[header]:
            if(dtMap[header_list.index(header)]):
                id = cell
            else:
                temp = "{}{}".format(header, cell)
                if(temp in convertMap):
                    id = convertMap[temp]
                else:
                    id = searchID(protagonist == header, cell, header)
                    convertMap[temp] = id
            columnList.append(id)
        print("finished processing {} datas".format(len(columnList)))
        if(header == protagonist):
            finalMap[header] = columnList
        else:
            finalMap[mapping[header.lower()]] = columnList

print("All task Finished")

Start processing Table
Processing Nama Sekolah Column
finished processing 25 datas
Processing Alamat Column
finished processing 25 datas
Processing Kelurahan Column
finished processing 25 datas
Processing Kecamatan Column
finished processing 25 datas
Processing Jumlah Siswa Column
finished processing 25 datas
Processing Telp Sekolah Column
finished processing 25 datas
Processing Lokasi Geografis Column
finished processing 25 datas
All task Finished


In [38]:
len(convertMap)

40

In [39]:
temp = {}
for item in convertMap:
    if(convertMap[item] == 'QNew' or convertMap[item] == 'QNPNew'):
        item2 = item.replace(' ', '').lower()
        temp[item2] = convertMap[item]
temp

{'kecamatankoja': 'QNPNew'}

In [43]:
new_df = pd.DataFrame.from_dict(finalMap)
new_df

,Nama Sekolah,P6375,P131,P131(2),P2196,P1329,P625
0,QNew,Jl. Bendera Raya - Intisari 3,Q12488768,Q193374,350,021-8720108,"-6.331491, 106.858711"
1,QNew,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Q7999779,Q192723,30,82114486336,"-6.161085, 106.773727"
2,QNew,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Q5058847,Q192714,60,(021)5447107,"-6.141474, 106.725876"
3,Q31778996,Jl. Kebon Nanas Utara II/23,Q12479264,Q193323,295,021-85914453,"1.000000, 2.000000"
4,QNew,Jl. Tabah Raya Komp. TNI AL,Q2806467,Q193545,512,(021)4535140,"-6.158968, 106.885063"
5,QNew,Jl. Let. Jend. S. Parman No. 69,Q13097193,Q192745,0,(021)70624646,"-6.193888, 106.797432"
6,QNew,Jl. Keutamaan No. 89,Q12492672,Q192748,144,(021) 6344508,"-6.156000, 106.814003"
7,Q60610068,Jl. Mindi No. 36,Q6472418,QNPNew,55,43909933,"-6.113743, 106.916031"
8,Q20427513,Jl. Kesatuan No.1 kp.Klingkit,Q4201202,Q192714,306,(021)58300174,"-6.156400, 106.728104"
9,Q12510798,Jl. Bendera Raya / Intisari III No.17,Q12488768,Q193374,209,(021)8728766,"-6.330335, 106.858643"


In [98]:
new_df.to_csv('data/dummymapping.csv')